In [192]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import xlwings as xw

In [184]:
books=[]
for i in range(1,51): #loop through the all pages
  url = f"https://books.toscrape.com/catalogue/page-{i}.html" #Access each page of the site
  reque = requests.get(url) 
  reque = reque.content #bring the structure of the page
  soup = BeautifulSoup(reque, 'html.parser')
  orderlist = soup.find('ol')
  articles = orderlist.find_all('article', class_='product_pod')
  for ar in articles:
    image = ar.find('img')
    title = image.attrs['alt'] #titles
    star = ar.find('p')
    star = star['class'][1] #star-rating
    price = ar.find('p', class_='price_color').text #price in text format
    price = float(price[1:]) #text to float
    books.append([title, star, price])

In [190]:
dados=pd.DataFrame(books,columns=['title','star rating','price'])

In [191]:
dados.to_csv('data_book.csv')

So, let's explore and see what we can do

In [193]:
dados.head()

,title,star rating,price
0,A Light in the Attic,Three,51.77
1,Tipping the Velvet,One,53.74
2,Soumission,One,50.10
3,Sharp Objects,Four,47.82
4,Sapiens: A Brief History of Humankind,Five,54.23


We are going to build the follow views:

--The top 5 expensives books



--The top 5 books in each star classification


--The accumulated price in each classification 

In [202]:
top_5_books=dados[['title','price']]
top_5_books=top_5_books.nlargest(columns='price',n=5)
top_5_books=top_5_books.reset_index(drop=True)

In [203]:
top_5_books.head()

,title,price
0,The Perfect Play (Play by Play #1),59.99
1,Last One Home (New Beginnings #1),59.98
2,Civilization and Its Discontents,59.95
3,The Barefoot Contessa Cookbook,59.92
4,The Diary of a Young Girl,59.90


In [242]:
accumulated_price=dados[['price','star rating']]
accumulated_price=accumulated_price.groupby(by='star rating').aggregate('sum').sort_values(by='price',ascending=False)
accumulated_price

,price
star rating,
One,7810.83
Three,7042.48
Five,6933.40
Two,6822.94
Four,6460.70


In [218]:
top5_one=dados[dados['star rating']=='One'].nlargest(n=5,columns='price').reset_index(drop=True)[['title','price']]
top5_one

,title,price
0,Thomas Jefferson and the Tripoli Pirates: The ...,59.64
1,The Improbability of Love,59.45
2,"Unstuffed: Decluttering Your Home, Mind, and Soul",58.09
3,"Miracles from Heaven: A Little Girl, Her Journ...",57.83
4,The Girl Who Kicked the Hornet's Nest (Millenn...,57.48


In [219]:
top5_two=dados[dados['star rating']=='Two'].nlargest(n=5,columns='price').reset_index(drop=True)[['title','price']]
top5_two

,title,price
0,Civilization and Its Discontents,59.95
1,Miller's Valley,58.54
2,The Lover's Dictionary,58.09
3,The Stand,57.86
4,Me Talk Pretty One Day,57.60


In [220]:
top5_three=dados[dados['star rating']=='Three'].nlargest(n=5,columns='price').reset_index(drop=True)[['title','price']]
top5_three

,title,price
0,The Perfect Play (Play by Play #1),59.99
1,Last One Home (New Beginnings #1),59.98
2,The Diary of a Young Girl,59.90
3,The Bone Hunters (Lexy Vaughan & Steven Macaul...,59.71
4,Boar Island (Anna Pigeon #19),59.48


In [221]:
top5_four=dados[dados['star rating']=='Four'].nlargest(n=5,columns='price').reset_index(drop=True)[['title','price']]
top5_four

,title,price
0,The Man Who Mistook His Wife for a Hat and Oth...,59.45
1,The Gray Rhino: How to Recognize and Act on th...,59.15
2,Unlimited Intuition Now,58.87
3,Myriad (Prentor #1),58.75
4,The Rose & the Dagger (The Wrath and the Dawn #2),58.64


In [222]:
top5_five=dados[dados['star rating']=='Five'].nlargest(n=5,columns='price').reset_index(drop=True)[['title','price']]
top5_five

,title,price
0,The Barefoot Contessa Cookbook,59.92
1,Life Without a Recipe,59.04
2,Approval Junkie: Adventures in Caring Too Much,58.81
3,How to Speak Golf: An Illustrated Guide to Lin...,58.32
4,Digital Fortress,58.00


Now, we are going to insert these table into excel file and build a dashboard

In [243]:
sheet_df_mapping={'aux':top_5_books}


with xw.App(visible=False) as app:
    wb = app.books.open('C:/Users/USER/OneDrive/Documentos/Abroad project/report_books.xlsx')
    # List of current worksheet names
    current_sheets = [sheet.name for sheet in wb.sheets] 
    # Iterate over sheet/df mapping 
    # If sheet already exist, overwrite current cotent. Else, add new sheet
    for sheet_name in sheet_df_mapping.keys():
        if sheet_name in current_sheets:
            wb.sheets(sheet_name).range('B2').value = sheet_df_mapping.get(sheet_name)
        else:
            new_sheet = wb.sheets.add(after=wb.sheets.count)
            new_sheet.range('B2').value = sheet_df_mapping.get(sheet_name)
            new_sheet.name = sheet_name
    wb.save()

In [244]:
sheet_df_mapping={'aux':accumulated_price}


with xw.App(visible=False) as app:
    wb = app.books.open('C:/Users/USER/OneDrive/Documentos/Abroad project/report_books.xlsx')
    # List of current worksheet names
    current_sheets = [sheet.name for sheet in wb.sheets] 
    # Iterate over sheet/df mapping 
    # If sheet already exist, overwrite current cotent. Else, add new sheet
    for sheet_name in sheet_df_mapping.keys():
        if sheet_name in current_sheets:
            wb.sheets(sheet_name).range('F2').value = sheet_df_mapping.get(sheet_name)
        else:
            new_sheet = wb.sheets.add(after=wb.sheets.count)
            new_sheet.range('F2').value = sheet_df_mapping.get(sheet_name)
            new_sheet.name = sheet_name
    wb.save()

In [245]:
sheet_df_mapping={'aux': top5_one}


with xw.App(visible=False) as app:
    wb = app.books.open('C:/Users/USER/OneDrive/Documentos/Abroad project/report_books.xlsx')
    # List of current worksheet names
    current_sheets = [sheet.name for sheet in wb.sheets] 
    # Iterate over sheet/df mapping 
    # If sheet already exist, overwrite current cotent. Else, add new sheet
    for sheet_name in sheet_df_mapping.keys():
        if sheet_name in current_sheets:
            wb.sheets(sheet_name).range('I2').value = sheet_df_mapping.get(sheet_name)
        else:
            new_sheet = wb.sheets.add(after=wb.sheets.count)
            new_sheet.range('I2').value = sheet_df_mapping.get(sheet_name)
            new_sheet.name = sheet_name
    wb.save()

In [246]:
sheet_df_mapping={'aux': top5_two}


with xw.App(visible=False) as app:
    wb = app.books.open('C:/Users/USER/OneDrive/Documentos/Abroad project/report_books.xlsx')
    # List of current worksheet names
    current_sheets = [sheet.name for sheet in wb.sheets] 
    # Iterate over sheet/df mapping 
    # If sheet already exist, overwrite current cotent. Else, add new sheet
    for sheet_name in sheet_df_mapping.keys():
        if sheet_name in current_sheets:
            wb.sheets(sheet_name).range('M2').value = sheet_df_mapping.get(sheet_name)
        else:
            new_sheet = wb.sheets.add(after=wb.sheets.count)
            new_sheet.range('M2').value = sheet_df_mapping.get(sheet_name)
            new_sheet.name = sheet_name
    wb.save()

In [247]:
sheet_df_mapping={'aux': top5_three}


with xw.App(visible=False) as app:
    wb = app.books.open('C:/Users/USER/OneDrive/Documentos/Abroad project/report_books.xlsx')
    # List of current worksheet names
    current_sheets = [sheet.name for sheet in wb.sheets] 
    # Iterate over sheet/df mapping 
    # If sheet already exist, overwrite current cotent. Else, add new sheet
    for sheet_name in sheet_df_mapping.keys():
        if sheet_name in current_sheets:
            wb.sheets(sheet_name).range('O2').value = sheet_df_mapping.get(sheet_name)
        else:
            new_sheet = wb.sheets.add(after=wb.sheets.count)
            new_sheet.range('O2').value = sheet_df_mapping.get(sheet_name)
            new_sheet.name = sheet_name
    wb.save()

In [248]:
sheet_df_mapping={'aux': top5_four}


with xw.App(visible=False) as app:
    wb = app.books.open('C:/Users/USER/OneDrive/Documentos/Abroad project/report_books.xlsx')
    # List of current worksheet names
    current_sheets = [sheet.name for sheet in wb.sheets] 
    # Iterate over sheet/df mapping 
    # If sheet already exist, overwrite current cotent. Else, add new sheet
    for sheet_name in sheet_df_mapping.keys():
        if sheet_name in current_sheets:
            wb.sheets(sheet_name).range('S2').value = sheet_df_mapping.get(sheet_name)
        else:
            new_sheet = wb.sheets.add(after=wb.sheets.count)
            new_sheet.range('S2').value = sheet_df_mapping.get(sheet_name)
            new_sheet.name = sheet_name
    wb.save()

In [249]:
sheet_df_mapping={'aux': top5_five}


with xw.App(visible=False) as app:
    wb = app.books.open('C:/Users/USER/OneDrive/Documentos/Abroad project/report_books.xlsx')
    # List of current worksheet names
    current_sheets = [sheet.name for sheet in wb.sheets] 
    # Iterate over sheet/df mapping 
    # If sheet already exist, overwrite current cotent. Else, add new sheet
    for sheet_name in sheet_df_mapping.keys():
        if sheet_name in current_sheets:
            wb.sheets(sheet_name).range('W2').value = sheet_df_mapping.get(sheet_name)
        else:
            new_sheet = wb.sheets.add(after=wb.sheets.count)
            new_sheet.range('W2').value = sheet_df_mapping.get(sheet_name)
            new_sheet.name = sheet_name
    wb.save()